In [ ]:
import pandas as pd
wiki=pd.read_csv('../input/wiki-rfa/wikiRfA.csv')
wiki.head()

In [ ]:
wiki.skew()

In [ ]:
wiki.nunique()

In [ ]:
wiki.info()

In [ ]:
round(wiki.SOURCE.value_counts()*100 / len(wiki),2)

In [ ]:
round(wiki.TARGET.value_counts()*100 / len(wiki),2)

In [ ]:
round(wiki.RESULT.value_counts()*100 / len(wiki),2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figsize=(10,5)
sns.boxplot(y=wiki['VOTE'])
plt.show()

In [ ]:
!pip install pandas-profiling

In [ ]:
from pandas_profiling import ProfileReport
ProfileReport(wiki, title="EDA Report")

In [ ]:
!pip install pycaret

In [ ]:
data = wiki.sample(frac=0.8, random_state=42)
data_unseen = wiki.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
from pycaret.classification import *

In [ ]:
wiki= setup(data = data, target = 'RESULT', session_id=42,
                  normalize = True, 
                  transformation = True, 
                  log_experiment = True,
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #rop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,# all levels in categorical features below the threshold defined in rare_level_threshold param are combined together as a single level
                  numeric_imputation='median',
           ignore_features=['Unnamed: 0'],
           categorical_features = ['SOURCE', 'TARGET', 'VOTE'],
            high_cardinality_features=['SOURCE', 'TARGET'],
           fix_imbalance = True,
            train_size = 0.8
          )

In [ ]:
qda = create_model('qda')

In [ ]:
print(qda)

In [ ]:
tuned_qda = tune_model(qda,optimize = 'Recall')

In [ ]:
plot_model(tuned_qda, plot = 'auc')

In [ ]:
plot_model(tuned_qda, plot = 'pr')

In [ ]:
plot_model(tuned_qda, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_qda)

In [ ]:
predict_model(tuned_qda)

In [ ]:
unseen_predictions = predict_model(tuned_qda, data=data_unseen)
unseen_predictions.head()

In [ ]:
print("Confidence Score :   {}".format(round(unseen_predictions.Score.mean(),2)))#Confidence Score